In [2]:
import pymongo
import shutil
import hashlib
import os
import random
import datetime
import json
from pprint import pprint
con = pymongo.MongoClient('localhost',27017)
FacetDB = con['faceted_search']

In [19]:
print FacetDB.collection_names()
mo = FacetDB['facet_images']

[u'system.profile', u'sessions', u'system.indexes', u'facet_filters', u'facet_images']


In [4]:
###  Each folder once unpacked should have a JSON file and a matching JPG file
###  These original names should be stored in the DB perhaps, but I move them and we use a Sha256 as the new name
XtractPath = '/home/dagutman/devel/FacetedSearch_V2/upload/extract/'
XtractGrpID = 'fb67b10f73'

In [12]:
## API Modules used on the NODE server include move.js, parse_archive.js 
##parse_files.js
! ls /home/dagutman/devel/FacetedSearch_V2/upload/extract/fb67b10f73/

ISIC_MSK-1_1  ISIC_MSK-2_1  ISIC_SONIC_1  ISIC_UDA-1_1	ISIC_UDA-2_1


In [1]:
### Get List of Files that have been created and need to be imported

ImagesToIngest = []

WebArchivePath = '/home/dagutman/devel/FacetedSearch_V2/upload/images/'



LOGIC:
    
    If JSON FILE VALID, but NO MATCHING .JPG/.PNG/.TIFF:
        
        then:
            
            CHECK IF remoteURL is in JSON File , if so, user that!  
    

MAGIC TAG IN THE JSON FILE==  "remoteURL"



for root, dirs, files in os.walk( os.path.join(XtractPath,XtractGrpID)):
    for f in files:
        if f.endswith('.jpg'):
            CurImageInfo = {}
            CurImagePath =  os.path.join(root,f)
            CurJSONPath = CurImagePath.replace('.jpg','.json')
            if os.path.isfile(CurJSONPath):
                
                sha256Id = hashlib.sha256()
                
                CurImageInfo['ImagePath'] = CurImagePath
                CurImageInfo['JSONPath'] = CurJSONPath
                CurImageInfo['__v'] =  0
                sha256Id.update(CurImagePath)
                sha256Id.update(datetime.datetime.now().isoformat())
                #.update(datetime.datetime.now()).update(random.random())
                    
                CurImageInfo['SHA256'] = sha256Id.hexdigest()
                
                CurImageInfo['filesrc'] =   { 'src' :'local', 'filePath': "somefilename.jpg"  }
                  Option B
                        {'src': "remote", 'remoteURL': "SomeURL"}
                
                sha256Id.hexdigest() + '.jpg'
                shutil.copy(CurImagePath, os.path.join(WebArchivePath,CurImageInfo['filename']))
                
                ### Actually load the JSON file
                try:
                    with open(CurJSONPath,'r') as jsfp:
                        JSONData = json.load(jsfp)
                    CurImageInfo['data'] = JSONData
                    TempDict = {}  ## This is needed because the extract function is recursive
                    FlattenedDict,FacetList = extract(JSONData,TempDict)
                    CurImageInfo['facets'] = FacetList

                    ImagesToIngest.append(CurImageInfo)
                    mo.insert_one(CurImageInfo)
                    pprint(CurImageInfo)
                    sys.exit()
#                 except:
#                     print "Unable to parse",CurJSONPath
            else:
                print "NO JSON found?"
                sys.exit()
        break
                
print len(ImagesToIngest)
# print ImagesToIngest[0]

SyntaxError: invalid syntax (<ipython-input-1-a6adff605bd9>, line 9)

In [18]:
ftl =  '/home/dagutman/devel/FacetedSearch_V2/upload/extract/fb67b10f73/ISIC_UDA-1_1/ISIC_0000450/ISIC_0000450.json'
with open(CurJSONPath,'r') as jsfp:
    JSONData = json.load(jsfp)
print JSONData

{u'license': u'', u'updated': u'2015-12-04T15:31:35.475000+00:00', u'_id': u'5436e3d3bae478396759f351', u'description': u'', u'largeImage': {u'sourceName': u'tiff', u'originalId': u'566031df9fc3c126e1f48a16', u'fileId': u'5436e3d3bae478396759f352'}, u'created': u'2014-10-09T19:36:51.692000+00:00', u'baseParentType': u'collection', u'name': u'ISIC_0000320', u'meta': {u'p1a_user': u'5450e996bae47865794e4d0d', u'p1b_result': u'ok', u'p1a_stop_time': u'2014-10-31T18:22:59.437000+00:00', u'p1b_stop_time': u'2015-12-04T15:31:40.807000+00:00', u'reviewed': {u'accepted': True, u'userId': u'5436c6e7bae4780a676c8f93', u'time': u'2014-11-10T02:39:56.492000+00:00'}, u'unstructured': {u'id1': u'324', u'site': u'mod', u'diagnosis': u'nevus nos', u'localization': u'Lower limb'}, u'p1b_start_time': u'2015-12-04T15:30:31.866000+00:00', u'clinical': {u'benign_malignant': u'benign', u'age_approx': 55, u'sex': u'male', u'diagnosis': u'nevus', u'diagnosis_confirm_type': None, u'melanocytic': True}, u'p1a_s

In [72]:
!ls /home/dagutman/devel/FacetedSearch_V2/upload/images/

3befae9084e02dbeb0a27545abd60f748cbb7f8f630b68a7d24c8335fbde9f32.jpg
57f47536e680107bbc3a28188d49b266e162458bcbb8e85980cb30039fb92328.jpg
5f9c485ea7997ddbf5fa5230b7ec24998ac0f18dd8d3cff760b6aa83b19d9d8d.jpg
62b49abad723f1a41b0f57ba7b8b4463c88327a378fecd05719723207fc0e604.jpg
65bac8d7ce092c7747fe7ff9aaa76d0671e6451d4bf6c599a8fe161a2b47abfa.jpg
740b98cac93fdf506795919ebc6d80eae77544b3d24f96cf556bccc043b49c32.jpg
a21126808fd8561d6ab7a14c6990b82b5c27f8405eff2c777df0c285228df94a.jpg
a3b555c36f11ffe1cc6a8b73d192c92206dba2e3114f96c0d2f63dc098f510a8.jpg
f763965799d28732b72e01a7da6534f3cadb764adac0424912d582346231e1c0.jpg


In [ ]:

# The file that you ask is this one: server/api/extensions/socket_backend.js, where are used the following files: 
# 1) server/api/extensions/parse_archive
# 2) server/api/extensions/parse_directory
# 3) server/api/extensions/parse_files 
# 4) server/api/extensions/create_facets

In [69]:
pprint(mo.find_one().keys())


sampleObj = mo.find_one()

sampleJSON = sampleObj['data']
sampleFacet = sampleObj['facets']
## Top Level properties include _id, data, __v presumably version, filename and facets

[u'facets', u'_id', u'data', u'__v', u'filename']


In [64]:


pprint(sampleObj)

{u'__v': 0,
 u'_id': ObjectId('58d961ae7f0c8f13967e8dde'),
 u'data': {u'_id': u'5592ac4a9fc3c13155a57a67',
           u'_modelType': u'item',
           u'baseParentId': u'55943cff9fc3c13155bcad5e',
           u'baseParentType': u'collection',
           u'created': u'2015-06-30T14:48:42.358000+00:00',
           u'creatorId': u'54cb974fbae47819d8e4c727',
           u'description': u'',
           u'folderId': u'5627f42b9fc3c132be08d84f',
           u'largeImage': {u'fileId': u'5592ac4a9fc3c13155a57a6a',
                           u'originalId': u'5592ac4a9fc3c13155a57a68',
                           u'sourceName': u'tiff'},
           u'license': u'',
           u'meta': {u'acquisition': {u'pixelsX': 6668, u'pixelsY': 4439},
                     u'clinical': {u'age_approx': 55,
                                   u'benign_malignant': u'malignant',
                                   u'diagnosis': u'basal cell carcinoma',
                                   u'diagnosis_confirm_type': u'hi

In [9]:
def extract(dict_in, dict_out):
    for key, value in dict_in.iteritems():
        if isinstance(value, dict): # If value itself is dictionary
            ### Need to update all the keys in value to include the key and insert a hash
            newD = {}
            for k,v in value.iteritems():
                ### Need to parse null values
                if isinstance(v,unicode):  #                 if ( v.isinstance('string') ):
                    if v=="":
                        v= "no value"
                newD[key+'|'+k] = v
            value = newD
            extract(value, dict_out)
        else:  #         elif isinstance(value, unicode):
            # Write to dict_out
            if value=="":
                value= "no value"
            dict_out[key] = value
    facetList = []
    for k,v in dict_out.iteritems():
        facetList.append( {'id': k, 'value': v, 'name': k.split('|')[-1]})
    ### The FacetList is actually a list 
    return dict_out, facetList
# if (
#                     (is.string(value) && value.trim === "")
#                     || (!value && !is.boolean(value))
#                 ) {
#                     value = "no value";
#                 } else if (is.boolean(value)) {
#                     value = value + "";
#                 }

In [62]:
lookatME = {}
FlattenedDict,FacetList = extract(sampleJSON,lookatME)
#print FacetList
## #Only difference is how it parses booleans.. true/false vs True/False
for fl in FacetList:
    if fl not in sampleFacet:
        print fl
print sampleFacet


{'id': u'meta|reviewed|accepted', 'value': True, 'name': u'accepted'}
{'id': u'meta|clinical|melanocytic', 'value': False, 'name': u'melanocytic'}
[{u'name': u'size', u'value': 36662184, u'id': u'size'}, {u'name': u'license', u'value': u'no value', u'id': u'license'}, {u'name': u'updated', u'value': u'2015-08-25T13:29:31.339000+00:00', u'id': u'updated'}, {u'name': u'folderId', u'value': u'5627f42b9fc3c132be08d84f', u'id': u'folderId'}, {u'name': u'description', u'value': u'no value', u'id': u'description'}, {u'name': u'sourceName', u'value': u'tiff', u'id': u'largeImage|sourceName'}, {u'name': u'originalId', u'value': u'5592ac4a9fc3c13155a57a68', u'id': u'largeImage|originalId'}, {u'name': u'fileId', u'value': u'5592ac4a9fc3c13155a57a6a', u'id': u'largeImage|fileId'}, {u'name': u'created', u'value': u'2015-06-30T14:48:42.358000+00:00', u'id': u'created'}, {u'name': u'baseParentType', u'value': u'collection', u'id': u'baseParentType'}, {u'name': u'p1a_user', u'value': u'55d4cde89fc3c14